In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import torch
from sentence_transformers import SentenceTransformer

from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data.csv")
df['date'] = pd.to_datetime(df['date'])
print(df.head())

In [ ]:
# date,text,value
# 2024-01-01,"Экономика растёт благодаря инвестициям",10.2
# 2024-01-01,"Рост потребительского доверия",
# 2024-01-02,"ЦБ повысил ключевую ставку",
# 2024-01-03,"Снижение курса рубля",-0.5

In [ ]:
# Выбираем модель для векторизации (русский/английский)
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

# Векторизация текста
texts = df['text'].fillna("").tolist()
embeddings = model.encode(texts, show_progress_bar=True)

# Добавляем эмбеддинги как отдельную колонку
df['embedding'] = list(embeddings)

In [ ]:
# Создаём DataFrame только с датами и эмбеддингами
df_embeddings = df[['date', 'embedding']].copy()

# Усреднение эмбеддингов по дням
daily_embeddings = df_embeddings.groupby('date')['embedding'].apply(lambda x: np.mean(np.stack(x), axis=0)).reset_index()

In [ ]:
# Оставляем только числовые значения
df_numeric = df[['date', 'value']].drop_duplicates().set_index('date').asfreq('D')

# Объединяем
merged_df = pd.merge(daily_embeddings, df_numeric, on='date', how='left').set_index('date').asfreq('D')

In [ ]:
# Разделение на текстовые эмбеддинги и числовые признаки
X_text = np.vstack(merged_df['embedding'].values)  # shape: [T, D]
X_num = merged_df[['value']].values  # shape: [T, 1]

# Объединяем в один датасет
X_combined = np.hstack([X_text, X_num])  # shape: [T, D+1]

In [ ]:
# Удаляем пропущие значения
X_combined_clean = X_combined[~np.isnan(X_combined).any(axis=1)]

# Обучаем VAR
model_var = VAR(X_combined_clean)
results = model_var.fit(maxlags=10, ic='aic')

# Прогнозирование
forecast = results.forecast(y=X_combined_clean[-results.k_ar:], steps=5)
print("Прогноз:\n", forecast)

In [ ]:
# Рисуем графики
plt.figure(figsize=(10, 4))
plt.plot(merged_df.index[-20:], X_combined_clean[-20:, -1], label='Real')
future_dates = pd.date_range(start=merged_df.index[-1], periods=6, freq='D')[1:]
plt.plot(future_dates, forecast[:, -1], label='Predicted', linestyle='--')
plt.legend()
plt.title('VAR Forecast')
plt.grid(True)
plt.show()